In [2]:
#conda activate pyvizenv

In [3]:
#pip install yfinance

In [24]:
# Import Libraries
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import panel as pn
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
pn.extension('plotly')
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi import NewsApiClient
sid = SentimentIntensityAnalyzer()


In [2]:
# Laod .env environment
load_dotenv()


True

In [3]:
# Pull in API Key
api_key = os.getenv("glassnode_api")
type(api_key)


str

In [4]:
# Define crypto currencies to pull
crypto_list = ["BTC"]


In [5]:
# Define Price URLs
price_url = 'https://api.glassnode.com/v1/metrics/market/price_usd'



## BTC API Data Pull

In [6]:
# Price API Request
btc_price_res = requests.get(price_url,
                      params={'a': 'BTC',
                              'i': '24h',
                              'api_key': api_key})


# Convert price to Pandas Dataframe, set index to time and clean up file
btc_price_df = pd.read_json(btc_price_res.text, convert_dates=['t'])
btc_price_df.columns = ['Date', 'BTC Price']
btc_price_df.set_index('Date', inplace=True) 




## BTC Data Table Debugging Cells

In [7]:
btc_price_df

,BTC Price
Date,
2010-07-17,0.049510
2010-07-18,0.085840
2010-07-19,0.080800
2010-07-20,0.074733
2010-07-21,0.079210
2010-07-22,0.055945
2010-07-23,0.062283
2010-07-24,0.054540
2010-07-25,0.050500


## BTC Daily Changes

In [8]:
# Daily change in price column
btc_daily_price_pct_change = btc_price_df.pct_change(1)
btc_daily_price_pct_change.columns = ['BTC Daily Price Change']
btc_daily_price_pct_change = btc_daily_price_pct_change.dropna()
# btc_daily_price_pct_change
btc_daily_price_pct_change.tail()

,BTC Daily Price Change
Date,
2021-04-28,-0.003421
2021-04-29,-0.022534
2021-04-30,0.078729
2021-05-01,0.000627
2021-05-02,-0.019002


## BTC Data Aggregating & Cleaning

In [9]:
# Define all the different data frames into a list
btc_frames = [btc_price_df, btc_daily_price_pct_change]

# Concatenate all the dataframes into one
btc_data = pd.concat(btc_frames, axis=1, join="outer", ignore_index=False)
btc_data.tail()


,BTC Price,BTC Daily Price Change
Date,,
2021-04-28,54793.394530,-0.003421
2021-04-29,53558.662069,-0.022534
2021-04-30,57775.278152,0.078729
2021-05-01,57811.517528,0.000627
2021-05-02,56712.980417,-0.019002


# News API Keys 

In [10]:
# Read your api key environment variable
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

In [11]:
 # Fetch all the news about Facebook Libra
bitcoin_headlines = newsapi.get_everything(
    q="bitcoin", language="en", sort_by="publishedAt")
print(f"Total articles about Bitcoin: {bitcoin_headlines['totalResults']}")


Total articles about Bitcoin: 9337


In [12]:
 # Transform the response dictionary to a DataFrame
bitcoin_df = pd.DataFrame.from_dict(bitcoin_headlines["articles"])

bitcoin_df.head()

,author,content,description,publishedAt,source,title,url,urlToImage
0,C Boarding Group,Dogecoin is burning up the newsfeeds these day...,Dogecoin is burning up the newsfeeds these day...,2021-05-02T16:00:53Z,"{'id': None, 'name': 'Cboardinggroup.com'}",How to book a hotel with Dogecoin (plus a spec...,https://cboardinggroup.com/how-to-book-a-hotel...,https://cboardinggroup.com/wp-content/uploads/...
1,Pierluigi Paganini,A new cryptocurrency stealer dubbed WeSteal is...,The bold author of a new cryptocurrency steale...,2021-05-02T16:00:15Z,"{'id': None, 'name': 'Securityaffairs.co'}","WeSteal, a shameless commodity cryptocurrency ...",https://securityaffairs.co/wordpress/117452/ma...,https://i1.wp.com/securityaffairs.co/wordpress...
2,Daniel Dubrovsky,"Market sentiment struggled this past week, wit...",The US Dollar finished strong as the Dow Jones...,2021-05-02T16:00:00Z,"{'id': None, 'name': 'DailyFX'}","Markets Week Ahead: US Dollar, Dow Jones, Gold...",https://www.dailyfx.com/forex/fundamental/fore...,https://a.c-dn.net/b/2gFPtk/headline_shutterst...
3,Dean Takahashi,"Join GamesBeat Summit 2021, happening today! W...",Satoshi's Games has launching its Elixir Marke...,2021-05-02T15:45:31Z,"{'id': None, 'name': 'VentureBeat'}",Satoshi’s Games launches Elixir Marketplace fo...,https://venturebeat.com/2021/05/02/satoshis-ga...,https://venturebeat.com/wp-content/uploads/202...
4,Kessler Topaz Meltzer & Check LLP,"RADNOR, Pa., May 02, 2021 (GLOBE NEWSWIRE) -- ...","RADNOR, Pa., May 02, 2021 (GLOBE NEWSWIRE) -- ...",2021-05-02T15:35:00Z,"{'id': None, 'name': 'Yahoo Entertainment'}","Kessler Topaz Meltzer & Check, LLP Announces I...",https://finance.yahoo.com/news/kessler-topaz-m...,https://s.yimg.com/uu/api/res/1.2/Y5sl3G577Fvf...


In [13]:
# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

In [14]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [15]:
# For headlines:
btc_headlines, dates = get_headlines("bitcoin")

Fetching news about 'bitcoin'
******************************
retrieving news from: 2021-05-02 00:00:00
retrieving news from: 2021-05-01 00:00:00
retrieving news from: 2021-04-30 00:00:00
retrieving news from: 2021-04-29 00:00:00
retrieving news from: 2021-04-28 00:00:00
retrieving news from: 2021-04-27 00:00:00
retrieving news from: 2021-04-26 00:00:00
retrieving news from: 2021-04-25 00:00:00
retrieving news from: 2021-04-24 00:00:00
retrieving news from: 2021-04-23 00:00:00
retrieving news from: 2021-04-22 00:00:00
retrieving news from: 2021-04-21 00:00:00
retrieving news from: 2021-04-20 00:00:00
retrieving news from: 2021-04-19 00:00:00
retrieving news from: 2021-04-18 00:00:00
retrieving news from: 2021-04-17 00:00:00
retrieving news from: 2021-04-16 00:00:00
retrieving news from: 2021-04-15 00:00:00
retrieving news from: 2021-04-14 00:00:00
retrieving news from: 2021-04-13 00:00:00
retrieving news from: 2021-04-12 00:00:00
retrieving news from: 2021-04-11 00:00:00
retrieving news

In [16]:
# Consider using previously defined function(s):
def create_df(news, language):
    articles = []
    for article in news:
        try:
            text = article["content"]
            articles.append({
                "text": text,
            })
        except AttributeError:
            pass
    return pd.DataFrame(articles)



def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment
# The function above should retain dates already

In [17]:
btc_avg_sent = pd.DataFrame(
    {
        "btc_avg": headline_sentiment_summarizer_avg(btc_headlines),
    }
)

In [18]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
btc_avg_sent.index = pd.to_datetime(dates)

In [19]:
# Merge with BTC returns
btc_sentiments = btc_daily_price_pct_change.join(btc_avg_sent).dropna(how="any")

# Display data
display(btc_sentiments)

,BTC Daily Price Change,btc_avg
Date,,
2021-04-03,-0.029451,-0.053460
2021-04-04,0.019254,0.157620
2021-04-05,0.002484,0.083360
2021-04-06,-0.011126,0.015850
2021-04-07,-0.034752,-0.043950
2021-04-08,0.037384,0.034785
2021-04-09,-0.001305,0.161575
2021-04-10,0.026581,0.101945
2021-04-11,0.007245,0.194425


In [20]:
# Correlate the headlines' sentiment to returns
btc_sentiments.corr()

,BTC Daily Price Change,btc_avg
BTC Daily Price Change,1.00000,0.10331
btc_avg,0.10331,1.00000


In [21]:
# Define features data
X = btc_sentiments.copy()
X = X.drop(columns=["btc_avg"])
X.head()

,BTC Daily Price Change
Date,
2021-04-03,-0.029451
2021-04-04,0.019254
2021-04-05,0.002484
2021-04-06,-0.011126
2021-04-07,-0.034752


In [22]:
# Define target data
y = btc_sentiments["btc_avg"].values
y = y.reshape(-1, 1)
y[:5]

array([[-0.05346],
       [ 0.15762],
       [ 0.08336],
       [ 0.01585],
       [-0.04395]])

In [25]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [26]:
# Create the scaler instance
X_scaler = StandardScaler()

In [27]:
# Fit the scaler
X_scaler.fit(X_train)

StandardScaler()

In [28]:
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)